In [4]:
import cdsapi
import xarray as xr
import pygrib
import numpy as np
import pandas as pd
import time
import requests
import os
import xarray as xr


import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors

from geopy.geocoders import Nominatim
from datetime import datetime
import timezonefinder
from astral.sun import sun
from astral.location import LocationInfo

In [ ]:
#What a permaculture project need to know when they start

In [5]:


# select the years you want to download:
start_year = 1992
end_year = 2022
year_range = [i for i in range(start_year, end_year + 1)]
print(year_range)

# Use geopy to get the latitude and longitude of the city
geolocator = Nominatim(user_agent="permaculture-climate")
location = geolocator.geocode("Puebla de don Fadrique, Spain")
# Add a delay between requests
time.sleep(1)


[1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


In [ ]:
#Send API request and download data
c = cdsapi.Client()
try:
    data = c.retrieve("reanalysis-era5-single-levels-monthly-means",
    {"format": "grib",
     "product_type": "monthly_averaged_reanalysis_by_hour_of_day",
     "variable": ['10m_u_component_of_wind', '10m_v_component_of_wind', 
                '2m_temperature',
                'total_cloud_cover', 
                'total_precipitation',
                ],
    "area": [location.latitude + 1, 
             location.longitude - 1, 
             location.latitude - 1, 
             location.longitude + 1],  # North, West, South, East. 
    "year": year_range,
    "month": ['01', '02', '03',
           '04', '05', '06',
           '07', '08', '09',
           '10', '11', '12'],
    "time": ["00:00","01:00","02:00","03:00","04:00","05:00",
             "06:00","07:00","08:00","09:00","10:00","11:00",
             "12:00", "13:00","14:00","15:00","16:00","17:00",
             "18:00","19:00","20:00","21:00","22:00","23:00"]
    })

    # Get the location of the file to download
    url = data.location

    # Download the file
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()

except requests.exceptions.HTTPError as errh:
    print ("HTTP Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
except requests.exceptions.RequestException as err:
    print ("Something went wrong with the request:",err)

else:
    # If the request was successful, write the file
    filename = 'download.grib'
    with open(filename, 'wb') as f:
        f.write(response.content)

    # Print the location where the file is saved
    print(f"File saved at: {os.path.abspath(filename)}")

In [2]:
# List of variables to load
variables = ['2t','10v','10u','tp','tcc']

# Dictionary to hold the datasets
datasets = {}

# Open the GRIB file for each variable
for var in variables:
    ds = xr.open_dataset('download.grib', engine='cfgrib', backend_kwargs={'filter_by_keys': {'shortName': var}})
    datasets[var] = ds

# Print the datasets
for var, ds in datasets.items():
    print(f"{var}:")
    print(ds)
    print("\n")

2t:
<xarray.Dataset>
Dimensions:     (time: 8928, latitude: 9, longitude: 9)
Coordinates:
    number      int32 ...
  * time        (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 38.96 38.71 38.46 38.21 ... 37.46 37.21 36.96
  * longitude   (longitude) float64 -3.435 -3.185 -2.935 ... -1.934 -1.684 358.6
    valid_time  (time) datetime64[ns] ...
Data variables:
    t2m         (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2023-12-04T09:10 GRIB to CDM+CF via cfgrib-0.9.1...


10v:
<xarray.Dataset>
Dimensions:     (time: 8928, latitude: 9, longitude: 9)
Coordi

In [4]:
# Check if the time values for Total precipitation are always 6 and 18
time_values = pd.to_datetime(datasets['tcc']['time'].values)
valid_times = [time for time in time_values if time.hour not in [6, 18]]

# Print the result
if valid_times:
    print(f"The following time values for Total precipitation are not 6 or 18: {valid_times}")
else:
    print("All time values for Total precipitation are 6 or 18.")

len(time_values)


The following time values for Total precipitation are not 6 or 18: [Timestamp('1992-01-01 00:00:00'), Timestamp('1992-01-01 01:00:00'), Timestamp('1992-01-01 02:00:00'), Timestamp('1992-01-01 03:00:00'), Timestamp('1992-01-01 04:00:00'), Timestamp('1992-01-01 05:00:00'), Timestamp('1992-01-01 07:00:00'), Timestamp('1992-01-01 08:00:00'), Timestamp('1992-01-01 09:00:00'), Timestamp('1992-01-01 10:00:00'), Timestamp('1992-01-01 11:00:00'), Timestamp('1992-01-01 12:00:00'), Timestamp('1992-01-01 13:00:00'), Timestamp('1992-01-01 14:00:00'), Timestamp('1992-01-01 15:00:00'), Timestamp('1992-01-01 16:00:00'), Timestamp('1992-01-01 17:00:00'), Timestamp('1992-01-01 19:00:00'), Timestamp('1992-01-01 20:00:00'), Timestamp('1992-01-01 21:00:00'), Timestamp('1992-01-01 22:00:00'), Timestamp('1992-01-01 23:00:00'), Timestamp('1992-02-01 00:00:00'), Timestamp('1992-02-01 01:00:00'), Timestamp('1992-02-01 02:00:00'), Timestamp('1992-02-01 03:00:00'), Timestamp('1992-02-01 04:00:00'), Timestamp('199

8928

In [9]:
# Calculate the climatology and average over latitude and longitude

avg_tp = datasets['tp']['tp'].groupby('time.month').mean(['time', 'latitude', 'longitude', 'step'])*1000
avg_temp = datasets['2t']['t2m'].groupby('time.month').mean(['time', 'latitude', 'longitude'])-273.15
max_temp = datasets['2t']['t2m'].groupby('time.month').max(['time', 'latitude', 'longitude'])-273.15
min_temp = datasets['2t']['t2m'].groupby('time.month').min(['time', 'latitude', 'longitude'])-273.15


#check the diemsnsions of the data
print(avg_temp.dims)
print(avg_tp.dims)



('month',)
('month',)


In [13]:


# Create a subplot with shared x-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add a bar chart for precipitation to the secondary y-axis
fig.add_trace(
    go.Bar(x=avg_tp.month, y=avg_tp, name='Precipitation', opacity=0.5),
    secondary_y=False,
)

# Add a line chart for temperature to the primary y-axis
fig.add_trace(
    go.Scatter(x=avg_temp.month, y=avg_temp, mode='lines', name='Temperature'),
    secondary_y=True,
)

# Set the layout to have two y-axes
fig.update_layout(
    title = 'Average Temperature and Precipitation',
    yaxis=dict(title='Precipitation (mm)'),
    yaxis2=dict(title='Temperature (°C)', overlaying='y', side='right'),
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=avg_temp.month,
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)

# Show the figure
fig.show()

In [14]:
#do do: revisit the title

print(plotly.__version__)

# Create a DataFrame from the DataArrays
df = pd.DataFrame({
    'month': avg_temp.month.values,
    'avg_temp': avg_temp.values,
    'max_temp': max_temp.values,
    'min_temp': min_temp.values
})

# Create a line chart for average temperature
fig = px.line(df, x='month', y='avg_temp', title='Min and max temparture')

# Add a line chart for max temperature
fig.add_trace(go.Scatter(x=df['month'], y=df['max_temp'], mode='lines', name='Max Temperature', line_color='red'))

# Add a line chart for min temperature
fig.add_trace(go.Scatter(x=df['month'], y=df['min_temp'], mode='lines', name='Min Temperature', line_color='red', fill='tonexty'))

# Add a line chart for min temperature
fig.add_hline(y=0, opacity=1, line_width=2, line_dash='dash', line_color='blue', annotation_text='freezing', annotation_position='top')

# Set the layout
fig.update_layout(
    yaxis=dict(title='Temperature (°C)'),
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=df['month'],
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)

# Show the figure
fig.show()


5.18.0


In [16]:
avg_u = datasets['10u']['u10'].groupby('time.month').mean(['latitude', 'longitude'])
avg_v = datasets['10v']['v10'].groupby('time.month').mean(['latitude', 'longitude'])

In [17]:

# Calculate wind speed
wind_speed = np.sqrt(avg_u**2 + avg_v**2)
#convert to km/h
wind_speed = wind_speed*3.6

# Calculate wind direction
wind_direction = np.arctan2(avg_v, avg_u) * (180 / np.pi)

# Adjust wind direction to be between 0 and 360 degrees
wind_direction = (wind_direction + 360) % 360

# Print the wind speed and direction
print("Wind Speed:")
print(wind_speed)
print("\n")
print("Wind Direction:")
print(wind_direction)


Wind Speed:
<xarray.DataArray (time: 8928)>
array([2.7199543, 2.6942606, 2.8063412, ..., 5.2000647, 5.3631635,
       5.258852 ], dtype=float32)
Coordinates:
    number      int32 0
  * time        (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00
    step        timedelta64[ns] 00:00:00
    surface     float64 0.0
    valid_time  (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00


Wind Direction:
<xarray.DataArray 'v10' (time: 8928)>
array([227.26468 , 231.01352 , 233.51729 , ...,  23.114014,  21.815002,
        19.27002 ], dtype=float32)
Coordinates:
    number      int32 0
  * time        (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00
    step        timedelta64[ns] 00:00:00
    surface     float64 0.0
    valid_time  (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00


In [18]:
#prepare the data for the wind rose

df = pd.DataFrame({'speed': wind_speed, 'direction': wind_direction})

df['direction'] = pd.cut(df['direction'], bins=np.linspace(0, 360, 9), labels=["N", "NE", "E", "SE", "S", "SW", "W", "NW"])
df['speed'] = pd.cut(df['speed'], bins=np.arange(0, df['speed'].max() + 1.1, 4))

# Calculate frequencies
frequency_df = df.groupby(['direction', 'speed']).size().reset_index(name='frequency')

# Calculate total frequency
total_frequency = frequency_df['frequency'].sum()

# Convert frequency to proportion
frequency_df['frequency'] = frequency_df['frequency'] / total_frequency

C:\Users\matthias\AppData\Local\Temp\ipykernel_16100\3757060769.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [19]:



# Get the number of unique 'speed' categories
num_categories = len(frequency_df['speed'].unique())

# Create a custom color scale with the same number of colors as there are categories
custom_color_scale = plt.cm.viridis_r(np.linspace(0, 4, num_categories))

# Convert the color scale to a list of hex color strings
custom_color_scale = [matplotlib.colors.rgb2hex(color) for color in custom_color_scale]

# Define a color map for the sorted 'speed' categories
color_map = {category: color for category, color in zip(sorted_categories, custom_color_scale)}

# Create the wind rose chart
fig = px.bar_polar(frequency_df, 
                   r='frequency', 
                   theta='direction', 
                   color='speed', 
                   template='simple_white', 
                   color_discrete_map=color_map)  # Use the color map

# Update the layout to make it rectangular
fig.update_layout(
    width=800,  # Set the width to 700 pixels
    height=600,  # Set the height to 1000 pixels
    polar_radialaxis_showgrid=True,  # Show radial grid
    polar_angularaxis_showgrid=True  # Show angular grid
)

fig.show()


NameError: name 'sorted_categories' is not defined

In [20]:
# Calculate the temperature difference
temp_diff = max_temp - min_temp

# Create a Plotly figure
fig = go.Figure()

# Add a line chart for temperature difference
fig.add_trace(go.Scatter(
    x=temp_diff.month,
    y=temp_diff,
    mode='lines',
    name='Temperature Difference'
))

# Set the layout
fig.update_layout(
    xaxis=dict(title='Month'),
    yaxis=dict(title='Temperature Difference (°C)'),
    title='Temperature Difference Data'
)

# Show the figure
fig.show()

In [21]:
# Create a Plotly figure
fig = go.Figure()

# Add a bar chart for min-max temperature
fig.add_trace(go.Bar(
    x=min_temp.month,
    y=max_temp - min_temp,
    base=min_temp,
    name='Min-Max Temperature'
))

# Set the layout
fig.update_layout(
    xaxis=dict(title='Month'),
    yaxis=dict(title='Temperature (°C)'),
    title='Min-Max Temperature Data'
)

# Show the figure
fig.show()

In [22]:
# Subtract the average temperature from min and max temperatures
min_temp_adj = min_temp - avg_temp
max_temp_adj = max_temp - avg_temp

# Create a Plotly figure
fig = go.Figure()

# Add a bar chart for adjusted min-max temperature
fig.add_trace(go.Bar(
    x=min_temp_adj.month,
    y=max_temp_adj - min_temp_adj,
    base=min_temp_adj,
    name='Adjusted Min-Max Temperature'
))

# Set the layout
fig.update_layout(
    xaxis=dict(title='Month'),
    yaxis=dict(title='Temperature Difference from Average (°C)'),
    title='Adjusted Min-Max Temperature Data'
)

# Show the figure
fig.show()

## Average cloud cover and sunrise/sunset times

In [23]:
#Get rid of the latitude and longitude dimensions by averaging the data
avg_tcc = ds['tcc'].mean(dim=['longitude', 'latitude'])


#Now average the data of each hour of each month across the 30 years of data. We end up with 288 data points, representing 24 h per month
month_hour_grouped = avg_tcc.groupby(avg_tcc['time.month'] * 100 + avg_tcc['time.hour'])
hourly_mean_by_month = month_hour_grouped.mean(dim='time')

print(avg_tcc)

<xarray.DataArray 'tcc' (time: 8928)>
array([0.44535828, 0.4701724 , 0.49484572, ..., 0.5452655 , 0.5927201 ,
       0.6065047 ], dtype=float32)
Coordinates:
    number      int32 ...
  * time        (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
    valid_time  (time) datetime64[ns] ...


In [31]:
#find the timezone of the location
tf = timezonefinder.TimezoneFinder()
timezone_str = tf.certain_timezone_at(lat=location.latitude, lng=location.longitude)

#define location infos for the astral package only using coordinates
location_info = LocationInfo(None, None, timezone_str, location.latitude, location.longitude)

#define two empty lists for sunrise and sunset times
sunrise_times, sunset_times = [], []

# append sunrise and sunset times for the 15th of every month of 2022. Automatically adjusted for Daylight Saving Time (DST)
for month in range(1, 13):
    date = datetime(2022, month, 15)
    
    s = sun(location_info.observer, date=date, tzinfo=timezone_str)
    sunrise_times.append(s['sunrise'].strftime('%H:%M'))
    sunset_times.append(s['sunset'].strftime('%H:%M'))

print(sunrise_times, sunset_times)
    

['08:24', '08:00', '07:21', '07:34', '06:59', '06:46', '06:59', '07:25', '07:51', '08:18', '07:50', '08:18'] ['18:13', '18:48', '19:16', '20:45', '21:12', '21:33', '21:31', '21:02', '20:17', '19:32', '17:57', '17:51']


In [32]:
# Create a graph with cloud cover values plus sunrise and sunset times

# Get rid of the coding of the month/hour combination used for grouping before. 
month_values = hourly_mean_by_month['group'] // 100
hour_values = hourly_mean_by_month['group'] % 100

# Reshape the data to match the format expected by Plotly
data_reshaped = hourly_mean_by_month.values.reshape((12, 24)).T  # Use -1 to automatically infer the size




fig = go.Figure()

fig.add_trace(go.Heatmap(
    z=data_reshaped,
    x=list(range(12)),
    y=list(range(24)),
    xgap = 5,
    colorscale='gray_r',
    colorbar=dict(title="Cloud Cover"),
))

# Set x-axis tickvals and ticktext for each month
fig.update_xaxes(
    tickvals=list(range(len(month_values))),
    ticktext=[f"{month_name}" for month_name in ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]],
    tickmode='array',  # Use 'array' for custom tickvals and ticktext
    tickangle=-45,  # Rotate tick labels for better readability
)

# Set axis labels and title
fig.update_layout(
    title='Monthly hourly mean cloud cover with sunrise and sunset times',
    yaxis_title='Hour of the day',
    xaxis_title='Month',
)



# Set x-axis tickvals and ticktext for each day of the month


# Add a line for sunset times
fig.add_trace(go.Scatter(
    x=list(range(12)),
    y=[float('{:.2f}'.format(int(h) + int(m) / 60)) for h, m in [time.split(':') for time in sunset_times]],
    mode='lines',
    line=dict(color='rgb(150,0,255)', width=2),
    name='Sunset'
))

# Add a line for sunrise times
fig.add_trace(go.Scatter(
    x=list(range(12)),
    y=[float('{:.2f}'.format(int(h) + int(m) / 60)) for h, m in [time.split(':') for time in sunrise_times]],
    mode='lines',
    line=dict(color='rgb(255,65,0)', width=2),
    
    name='Sunrise'
))


# Update layout to show custom line in legend
fig.update_layout(
    showlegend=True,
    legend=dict(x=1.02, y=1.15),
    yaxis=dict(
        dtick=2,)
)


# Show the plot

fig.show()

# PROJECTION DATA

In [18]:
start_year = 2015
end_year = 2045
year_range_forecast = [str(i) for i in range(start_year, end_year + 1)]
print(year_range)

dataset_variables = ['air_temperature', 'eastward_near_surface_wind', 'northward_near_surface_wind', 'precipitation']
for variable in dataset_variables:
    print('download_' + variable + '.grib')

['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030', '2031', '2032', '2033', '2034', '2035', '2036', '2037', '2038', '2039', '2040', '2041', '2042', '2043', '2044', '2045']
download_air_temperature.grib
download_eastward_near_surface_wind.grib
download_northward_near_surface_wind.grib
download_precipitation.grib


In [29]:
c = cdsapi.Client()

dataset_variables = ['air_temperature', 'eastward_near_surface_wind', 'northward_near_surface_wind', 'precipitation']
for variable in dataset_variables:
    #create a loop as air temperature also needs 'level' while the others dont
    if variable == 'air_temperature':            
        data = c.retrieve(
            'projections-cmip6',
            {
                'format': 'zip',
                'temporal_resolution': 'monthly',
                'variable': variable,
                'experiment': 'ssp2_4_5',
                'model': 'cmcc_esm2',
                'area':[location.latitude + 1, 
                    location.longitude - 1, 
                    location.latitude - 1, 
                    location.longitude + 1],  # North, West, South, East. 
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'year': year_range_forecast,
                'level': '1000',
            },
            'download.zip')
        url = data.location

        # Download the file
        response = requests.get(url)

       # Check if the request was successful
        response.raise_for_status()
    else:         
        data = c.retrieve(
            'projections-cmip6',
            {
                'format': 'zip',
                'temporal_resolution': 'monthly',
                'variable': variable,
                'experiment': 'ssp2_4_5',
                'model': 'cmcc_esm2',
                'area':[location.latitude + 1, 
                    location.longitude - 1, 
                    location.latitude - 1, 
                    location.longitude + 1],  # North, West, South, East. 
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'year': year_range_forecast,
                
            },
            'download.zip')    
    # Get the location of the file to download
        url = data.location

        # Download the file
        response = requests.get(url)

        # Check if the request was successful
        response.raise_for_status()

    # If the request was successful, write the file
    filename = 'download_' + variable + '.grib'
    with open(filename, 'wb') as f:
        f.write(response.content)

        # Print the location where the file is saved
        print(f"File saved at: {os.path.abspath(filename)}")
print('Downloads completed')



2023-12-04 12:12:05,097 INFO Welcome to the CDS
2023-12-04 12:12:05,098 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-12-04 12:12:05,249 INFO Request is completed
2023-12-04 12:12:05,251 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data9/adaptor.esgf_wps.retrieve-1701685876.4409328-10818-12-494bff2c-5d9e-4fd3-845d-752e52dcf04b.zip to download.zip (210K)
2023-12-04 12:12:06,646 INFO Download rate 150.7K/s
2023-12-04 12:12:09,765 INFO Welcome to the CDS
2023-12-04 12:12:09,766 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-12-04 12:12:09,864 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data9/adaptor.esgf_wps.retrieve-1701687316.5899117-18839-11-d7ce93ce-5098-45e4-8ed5-dd480729fc30.zip to download.zip (269.6K)


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\download_air_temperature.grib


2023-12-04 12:12:11,371 INFO Download rate 179.2K/s


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\download_eastward_near_surface_wind.grib


2023-12-04 12:12:15,360 INFO Welcome to the CDS
2023-12-04 12:12:15,364 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2023-12-04 12:12:15,508 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data6/adaptor.esgf_wps.retrieve-1701688023.2072248-7226-15-17a0f3fa-7dbe-4491-9296-06abf8e45f4a.zip to download.zip (258.9K)
2023-12-04 12:12:18,309 INFO Download rate 92.5K/s
2023-12-04 12:12:29,606 INFO Welcome to the CDS
2023-12-04 12:12:29,608 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\download_northward_near_surface_wind.grib


2023-12-04 12:12:30,036 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data9/adaptor.esgf_wps.retrieve-1701687663.886994-10646-15-501ccc0a-32ad-43f1-8704-7cc6b52b7507.zip to download.zip (277.4K)
2023-12-04 12:12:32,777 INFO Download rate 101.2K/s


File saved at: c:\Users\matthias\Documents\Projects\permaculture-climate\download_precipitation.grib
Downloads completed
